# 🔐 Security Operations Log Analysis

> **Author:** Vandana Kumari  
> **Domain:** Cyber Security & Data Analytics  
> **Tools Used:** Python, Pandas, Jupyter Notebook  

### Project Objective  
To analyze authentication logs and prepare clean, structured data for a
Security Operations (SOC) dashboard.


## 📌 Project Overview

Organizations generate thousands of login records every day.  
Analyzing these logs manually is inefficient and error-prone.

This project focuses on cleaning and preparing authentication log data
to support a **Security Operations Dashboard** that helps identify:
- Suspicious login behavior
- High-risk users and IP addresses
- Time-based attack patterns


importing libraries

In [25]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r"C:\Users\Vanda\OneDrive\Desktop\security-operations-dashboard\security_operations_dataset.csv")

CLEANING DATASET

In [3]:
# shape of the dataset
df.shape

(5000, 8)

In [4]:
# column names
df.columns

Index(['user_id', 'login_time', 'login_status', 'ip_address', 'country',
       'device', 'os', 'browser'],
      dtype='object')

In [5]:
# Data types
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       5000 non-null   object
 1   login_time    5000 non-null   object
 2   login_status  5000 non-null   object
 3   ip_address    5000 non-null   object
 4   country       5000 non-null   object
 5   device        5000 non-null   object
 6   os            5000 non-null   object
 7   browser       5000 non-null   object
dtypes: object(8)
memory usage: 312.6+ KB


In [6]:
# chceking missing values
df.isnull().sum()

user_id         0
login_time      0
login_status    0
ip_address      0
country         0
device          0
os              0
browser         0
dtype: int64

## 🧹 Removing Duplicate Records

Duplicate login records can distort analysis and lead to incorrect security insights.
These duplicates are identified and removed.


In [7]:
# Check duplicates
df.duplicated().sum()

np.int64(0)

In [8]:
# Remove duplicates
df.drop_duplicates(inplace=True)

# Confirm removal
df.duplicated().sum()

np.int64(0)

In [9]:
# Convert login_time to datetime
df['login_time'] = pd.to_datetime(df['login_time'])

df.dtypes


user_id                 object
login_time      datetime64[ns]
login_status            object
ip_address              object
country                 object
device                  object
os                      object
browser                 object
dtype: object

In [10]:
# Drop rows with missing critical values
df.dropna(subset=[
    'user_id', 
    'login_time', 
    'login_status', 
    'ip_address'
], inplace=True)

# Check again
df.isnull().sum()


user_id         0
login_time      0
login_status    0
ip_address      0
country         0
device          0
os              0
browser         0
dtype: int64

In [11]:
# Standardize text columns
df['login_status'] = df['login_status'].str.strip().str.capitalize()
df['country'] = df['country'].str.strip().str.title()
df['device'] = df['device'].str.strip().str.capitalize()
df['os'] = df['os'].str.strip()
df['browser'] = df['browser'].str.strip()

df.head()


,user_id,login_time,login_status,ip_address,country,device,os,browser
0,user_164,2025-12-14 10:01:41,Success,80.125.114.71,Australia,Mobile,macOS,Chrome
1,user_152,2025-12-28 13:16:41,Success,33.111.119.13,Canada,Mobile,macOS,Safari
2,user_57,2025-12-29 19:04:41,Success,217.3.81.216,Uk,Desktop,Linux,Firefox
3,user_196,2025-12-24 16:04:41,Success,107.49.183.176,Canada,Desktop,Windows,Safari
4,user_138,2025-12-15 00:45:41,Failed,106.40.150.185,Canada,Mobile,Windows,Chrome


In [12]:
# Unique login status values
df['login_status'].unique()


array(['Success', 'Failed'], dtype=object)

In [13]:
# Keep only valid statuses
df = df[df['login_status'].isin(['Success', 'Failed'])]

df['login_status'].value_counts()


login_status
Success    3783
Failed     1217
Name: count, dtype: int64

In [14]:
df['date'] = df['login_time'].dt.date
df['hour'] = df['login_time'].dt.hour
df['day'] = df['login_time'].dt.day_name()
df['month'] = df['login_time'].dt.month_name()

df.head()


,user_id,login_time,login_status,ip_address,country,device,os,browser,date,hour,day,month
0,user_164,2025-12-14 10:01:41,Success,80.125.114.71,Australia,Mobile,macOS,Chrome,2025-12-14,10,Sunday,December
1,user_152,2025-12-28 13:16:41,Success,33.111.119.13,Canada,Mobile,macOS,Safari,2025-12-28,13,Sunday,December
2,user_57,2025-12-29 19:04:41,Success,217.3.81.216,Uk,Desktop,Linux,Firefox,2025-12-29,19,Monday,December
3,user_196,2025-12-24 16:04:41,Success,107.49.183.176,Canada,Desktop,Windows,Safari,2025-12-24,16,Wednesday,December
4,user_138,2025-12-15 00:45:41,Failed,106.40.150.185,Canada,Mobile,Windows,Chrome,2025-12-15,0,Monday,December


In [15]:
# Define suspicious hours (late night)
df['late_night_login'] = df['hour'].apply(
    lambda x: 1 if x >= 0 and x <= 4 else 0
)

df['late_night_login'].value_counts()


late_night_login
0    4004
1     996
Name: count, dtype: int64

In [16]:
# IP address length check
df['ip_address'].apply(len).describe()


count    5000.000000
mean       13.336800
std         1.101094
min         9.000000
25%        13.000000
50%        13.000000
75%        14.000000
max        15.000000
Name: ip_address, dtype: float64

In [17]:
# Users count
df['user_id'].nunique()


200

In [18]:
# Countries count
df['country'].nunique()


7

In [19]:
df.shape


(5000, 13)

In [23]:
import os

os.makedirs("data", exist_ok=True)
df.to_csv("data/cleaned_login_logs.csv", index=False)



In [26]:
df.head()

,user_id,login_time,login_status,ip_address,country,device,os,browser,date,hour,day,month,late_night_login
0,user_164,2025-12-14 10:01:41,Success,80.125.114.71,Australia,Mobile,macOS,Chrome,2025-12-14,10,Sunday,December,0
1,user_152,2025-12-28 13:16:41,Success,33.111.119.13,Canada,Mobile,macOS,Safari,2025-12-28,13,Sunday,December,0
2,user_57,2025-12-29 19:04:41,Success,217.3.81.216,Uk,Desktop,Linux,Firefox,2025-12-29,19,Monday,December,0
3,user_196,2025-12-24 16:04:41,Success,107.49.183.176,Canada,Desktop,Windows,Safari,2025-12-24,16,Wednesday,December,0
4,user_138,2025-12-15 00:45:41,Failed,106.40.150.185,Canada,Mobile,Windows,Chrome,2025-12-15,0,Monday,December,1
